In [5]:
import os

os.environ['KAGGLE_USERNAME'] = "hatitara"
os.environ['KAGGLE_KEY'] = "8076631b38e08331efbdfe56a0264de0"

import kagglehub

print("Downloading dataset...")
path = kagglehub.dataset_download("CooperUnion/anime-recommendations-database")

print("Path to dataset files:", path)
print(os.listdir(path))

Using Colab cache for faster access to the 'anime-recommendations-database' dataset.
Path to dataset files: /kaggle/input/anime-recommendations-database
['rating.csv', 'anime.csv']


In [6]:
!pip install -q -U torch transformers accelerate bitsandbytes fuzzywuzzy python-levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [8]:
import transformers
print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.57.3


In [9]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from fuzzywuzzy import proce

anime_file = f"{path}/anime.csv"
rating_file = f"{path}/rating.csv"

anime_df = pd.read_csv(anime_file)
ratings_df = pd.read_csv(rating_file)

anime_df.dropna(subset=['genre', 'type'], inplace=True)
# Keep map of Name -> ID for LLM mapping
name_to_id = pd.Series(anime_df.anime_id.values, index=anime_df.name).to_dict()
id_to_name = pd.Series(anime_df.name.values, index=anime_df.anime_id).to_dict()

# 2. Load Qwen 2.5-7B (4-bit Quantized)
model_id = "Qwen/Qwen1.5-MoE-A2.7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    dtype=torch.float32
)

print("Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

Model loaded successfully!


Function for Generating LLM Recommendations

The function below:
- Takes a list of anime titles a user has watched
- Sends them to Qwen inside a chat prompt
- Receives a comma-separated list of suggested anime titles
- Fuzzy-matches these titles to our Kaggle dataset
- Returns anime IDs

This is **not** evaluated using metrics — we only use it to display example output.

In [10]:
def get_llm_recommendations(user_history_names, k=10):
    history_str = ", ".join(user_history_names[-20:])

    prompt = [
        {"role": "system", "content": "You are an expert anime recommender. Reply ONLY with a list of anime titles separated by commas. Do not write numbered lists or explanations."},
        {"role": "user", "content": f"I have watched and liked these anime: {history_str}. Recommend {k} different anime that are similar but NOT in this list. Return only titles."}
    ]

    text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    raw_titles = [t.strip() for t in response.split(',')]

    rec_ids = []
    for title in raw_titles:
        match, score = process.extractOne(title, name_to_id.keys())
        if score > 80:
            rec_ids.append(name_to_id[match])

    return rec_ids[:k]

Example Output From Qwen Recommender

Below we provide a demonstration using a manually selected user history  
to show how the LLM responds.  
This is **not part of the evaluation pipeline** and is included for illustrative purposes.

In [11]:
example_history = [
    "Attack on Titan",
    "Death Note",
    "Fullmetal Alchemist: Brotherhood"
]

print("Input user history:")
print(example_history)

print("\nLLM-generated recommendations:")
demo = get_llm_recommendations(example_history, k=10)

demo_names = [id_to_name[i] for i in demo if i in id_to_name]

for name in demo_names:
    print(" -", name)


Input user history:
['Attack on Titan', 'Death Note', 'Fullmetal Alchemist: Brotherhood']

LLM-generated recommendations:
 - Code Geass: Hangyaku no Lelouch R2
 - Magi: The Kingdom of Magic


Find LLM recommendations inside our Kaggle anime dataset

In [12]:
recs_df = anime_df[anime_df["anime_id"].isin(demo)].copy()

recs_df = recs_df.set_index("anime_id").loc[[i for i in demo if i in recs_df.index]]

cols_to_show = ["name", "genre", "type", "episodes", "rating", "members"]

print("\nRecommendations found in our dataset:")
display(recs_df[cols_to_show])



Recommendations found in our dataset:


,name,genre,type,episodes,rating,members
anime_id,,,,,,
